Desenvolva um programa (em Phyton) que processe as seguintes sequências de bits:

1000000001010011

1110100101000010

Códigos: NRZ, B8ZS (Bipolar with Eight Zeros Substitution), Manchester diferencial.

In [ ]:
from matplotlib.pyplot import figure
from itertools import product

In [ ]:
def gen_img(funcname: str, sequence: int,
    dom: list[int], img: list[int]):
  fig = figure(None, (12, 2), 150)
  axe = fig.subplots()
  axe.set_title(f"{funcname} - {bin(sequence)}")
  axe.set_xlabel("Tempo")
  axe.set_ylabel("Voltagem")
  axe.step(dom, img, where = "post")
  axe.grid(True)
  fig.tight_layout(pad = 1)

In [ ]:
def nrz(data: int):
  dom = range(data.bit_length())
  img = [10 * bool(data & (1 << i)) - 5 for i in reversed(dom)]
  return list(dom), img

In [ ]:
def b8zs(data: int):
  dom = range(data.bit_length())
  def b8zs_core():
    ones, zeros = False, 0
    for i in reversed(dom):
      if data & (1 << i):
        yield from (([0] * zeros) + [10 * ones - 5])
        ones, zeros = not ones, 0
        continue
      zeros += 1
      if zeros == 8:
        if not ones:
          yield from (0, 0, 0, 5, -5, 0, -5, 5)
        else:
          yield from (0, 0, 0, -5, 5, 0, 5, -5)
        zeros = 0
    yield from ([0] * zeros)
  return list(dom), list(b8zs_core())

In [ ]:
def dif_man(data: int):
  order = data.bit_length()
  dom = [e / 2 for e in range(order * 2)]
  def difman_core():
    prev = 0
    for i in reversed(range(order)):
      prev ^= bool(data & (1 << i))
      yield 10 * (not prev) - 5
      yield 10 * prev - 5
  return dom, list(difman_core())

In [ ]:
funcs = (nrz, b8zs, dif_man)
seqs = (0b1000000001010011, 0b1110100101000010)
for func, seq in product(funcs, seqs):
  dom, img = func(seq)
  gen_img(func.__name__, seq, dom, img)